In [ ]:
'''!pip install aicrowd-cli
API_KEY = "9967b67c796b4d50e5d26f4dfd97ed06" 
!aicrowd login --api-key $API_KEY
!aicrowd dataset download --challenge global-wheat-challenge-2021'''

'!pip install aicrowd-cli\nAPI_KEY = "9967b67c796b4d50e5d26f4dfd97ed06" \n!aicrowd login --api-key $API_KEY\n!aicrowd dataset download --challenge global-wheat-challenge-2021'

In [ ]:
'''import shutil
shutil.copyfile('/content/train.zip',
                '/content/drive/MyDrive/aicrowd_wheat/train.zip')
shutil.copyfile('/content/test.zip',
                '/content/drive/MyDrive/aicrowd_wheat/test.zip')
shutil.copyfile('/content/submission.csv',
                '/content/drive/MyDrive/aicrowd_wheat/submission.csv')'''

"import shutil\nshutil.copyfile('/content/train.zip',\n                '/content/drive/MyDrive/aicrowd_wheat/train.zip')\nshutil.copyfile('/content/test.zip',\n                '/content/drive/MyDrive/aicrowd_wheat/test.zip')\nshutil.copyfile('/content/submission.csv',\n                '/content/drive/MyDrive/aicrowd_wheat/submission.csv')"

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Jun 19 09:21:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install albumentations==0.4.6
!pip install ensemble-boxes

     |████████████████████████████████| 122kB 4.3MB/s 
     |████████████████████████████████| 952kB 33.0MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp37-none-any.whl size=65175 sha256=294e233700b15bfcdc0890f1ebffd2f4d4dee5e318d587343777777bb0fa9d55
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
import shutil
import pandas as pd
import numpy as np 
import os
import cv2
import re

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN  
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt
from datetime import datetime

from sklearn.model_selection import StratifiedKFold, train_test_split
import joblib
from ensemble_boxes import *

In [ ]:
if(os.path.isdir('test')):
  print('folder exists, skip unzip procedure')
else:
  shutil.copyfile('/content/drive/MyDrive/aicrowd_wheat/test.zip','/content/test.zip')
  shutil.unpack_archive('./test.zip')
  #!unzip train.zip

In [ ]:
shutil.copyfile('/content/drive/MyDrive/aicrowd_wheat/submission.csv','/content/submission.csv')

'/content/submission.csv'

In [ ]:
test_df = pd.read_csv('submission.csv')
test_df['image_id'] = test_df['image_name']
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2849 entries, 0 to 2848
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  2849 non-null   object
 1   domain      2849 non-null   int64 
 2   PredString  2849 non-null   object
 3   image_id    2849 non-null   object
dtypes: int64(1), object(3)
memory usage: 89.2+ KB


In [ ]:
test_df.head()

,image_name,domain,PredString,image_id
0,c087c8b89904edee56997a25d64f4cd1b7c9441e64889a...,18,no_box,c087c8b89904edee56997a25d64f4cd1b7c9441e64889a...
1,b11a617a64ca6120021e70b107933ddb70bf052395759d...,18,no_box,b11a617a64ca6120021e70b107933ddb70bf052395759d...
2,4fd7a4dc7dc60b5daedc7d57fdf8000ef3cab267654b56...,18,no_box,4fd7a4dc7dc60b5daedc7d57fdf8000ef3cab267654b56...
3,3f60fada17caf269a8d70798eb3bb5771f2c10d13c887e...,18,no_box,3f60fada17caf269a8d70798eb3bb5771f2c10d13c887e...
4,8cdcb28a926c217d0b587fcba2050e283b3175824a5b6e...,18,no_box,8cdcb28a926c217d0b587fcba2050e283b3175824a5b6e...


In [ ]:
len(os.listdir('./test/')), os.listdir('./test/')[0:3]

(2856,
 ['fa590cb92c7ab0ee96ff5f94c2fffc5f76c1f01786ba5197edd2670efbc39031.png',
  'd8d2e3dfeee9e138b87b7733827a9d60fb1e0661c04cf3cab4ec4e36ac3929f8.png',
  '06bc0e1772e40077458aca53c1c9af22c885ed23c0d4f7b4bd369567a0660a4a.png'])

In [ ]:
class WheatTestDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]

        image = cv2.imread(f'{self.image_dir}/{image_id}.png', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        if self.transforms:
            sample = {
                'image': image,
            }
            sample = self.transforms(**sample)
            image = sample['image']

        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
def get_test_transform():
    return A.Compose([
        # A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ])

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

In [ ]:
DIR_INPUT = './'
DIR_TRAIN = f'{DIR_INPUT}/train'
DIR_TEST = f'{DIR_INPUT}/test'

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 2  # 1 class (wheat) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load the trained weights
#model.load_state_dict(torch.load('/content/drive/MyDrive/aicrowd_wheat/'\
#                                 +'fasterrcnn_resnet50_epochs_40_lr_0.005_time_13_06_2021_21_23.pth'))
#### best model 0.604 score #####
#model.load_state_dict(torch.load('/content/drive/MyDrive/aicrowd_wheat/'\
#                                 +'fasterrcnn_resnet50_epochs_20_lr_0.005_time_15_06_2021_13_55.pth'))
#### best model 0.611 score #####
model.load_state_dict(torch.load('/content/drive/MyDrive/aicrowd_wheat/'\
                                 +'fasterrcnn_resnet50_epochs_10_lr_0.005_time_18_06_2021_08_23.pth'))
model.eval()
x = model.to(device)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

test_dataset = WheatTestDataset(test_df, DIR_TEST, get_test_transform())

test_data_loader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    #for j in zip(scores, boxes):
    #    pred_strings.append("{1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    if len(boxes) >0:
      boxes = [" ".join([str(int(i)) for i in item]) for item in boxes]
      BoxesString = ";".join(boxes)
    else:
      BoxesString = "no_box"
    # return ";".join(pred_strings)
    return BoxesString

In [ ]:
def format_scores_string(boxes, scores):
    score_strings = []
    #for j in zip(scores, boxes):
    #    pred_strings.append("{1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    if len(scores) >0:
      ScoreString = ";".join([str(i) for i in scores])
    else:
      ScoreString = "no_box"
    # return ";".join(pred_strings)
    return ScoreString

In [ ]:
from ensemble_boxes import *

def run_wbf(boxes,scores, image_size=1024, iou_thr=0.41, skip_box_thr=0.4, weights=None):
  # boxes=boxes/(image_size-1)
  labels0 = [[0 for idx in range(len(scores))]]
  boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels0, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
  # boxes=boxes*(image_size-1)
  # boxes = boxes
  return boxes, scores, labels

In [ ]:
df_binary = pd.read_csv('/content/drive/MyDrive/aicrowd_wheat/binary_preds.csv')
df_binary['image_name'] = df_binary['file_name'].str.replace('.png','')
df_binary['prediction'].value_counts()

yes    2785
no       71
Name: prediction, dtype: int64

In [ ]:
# detection_threshold = 0.6
results = []

for images, image_ids in test_data_loader:

    images = list(image.to(device) for image in images)
    outputs = model(images)

    for i, image in enumerate(images):

        boxes = outputs[i]['boxes'].data.cpu().numpy()
        scores = outputs[i]['scores'].data.cpu().numpy()

        # weighted boxes
        #boxes1, scores1, labels1 = weighted_boxes_fusion([boxes/boxes.max().tolist()], 
        #                                      [scores.tolist()],
        #                                      [[0 for i in range(boxes.shape[0])]],
        #                                      iou_thr=0.1, skip_box_thr=0.05)
        #boxes = boxes1*boxes.max()
        #scores = scores1
        
        #boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        #boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        image_id = image_ids[i]
        if(df_binary[df_binary['image_name']==image_id]['prediction'].values[0]=='no'):
          detection_threshold = 0.0
        else:
          detection_threshold = 0.6

        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        scores = scores[scores >= detection_threshold]
        result = {
            'image_id': image_id,
            'predictions': format_prediction_string(boxes, scores),
            'probabilities': format_scores_string(boxes, scores)
        }
      
        results.append(result)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
'''detection_threshold = 0.6
results = []

for images, image_ids in test_data_loader:

    images = list(image.to(device) for image in images)
    outputs = model(images)

    for i, image in enumerate(images):

        boxes = outputs[i]['boxes'].data.cpu().numpy()
        scores = outputs[i]['scores'].data.cpu().numpy()

        # weighted boxes
        #boxes1, scores1, labels1 = weighted_boxes_fusion([boxes/boxes.max().tolist()], 
        #                                      [scores.tolist()],
        #                                      [[0 for i in range(boxes.shape[0])]],
        #                                      iou_thr=0.1, skip_box_thr=0.05)
        #boxes = boxes1*boxes.max()
        #scores = scores1
        
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        scores = scores[scores >= detection_threshold]
        image_id = image_ids[i]
        
        #boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        #boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        result = {
            'image_id': image_id,
            'predictions': format_prediction_string(boxes, scores),
            'probabilities': format_scores_string(boxes, scores)
        }

        
        results.append(result)'''

"detection_threshold = 0.6\nresults = []\n\nfor images, image_ids in test_data_loader:\n\n    images = list(image.to(device) for image in images)\n    outputs = model(images)\n\n    for i, image in enumerate(images):\n\n        boxes = outputs[i]['boxes'].data.cpu().numpy()\n        scores = outputs[i]['scores'].data.cpu().numpy()\n\n        # weighted boxes\n        #boxes1, scores1, labels1 = weighted_boxes_fusion([boxes/boxes.max().tolist()], \n        #                                      [scores.tolist()],\n        #                                      [[0 for i in range(boxes.shape[0])]],\n        #                                      iou_thr=0.1, skip_box_thr=0.05)\n        #boxes = boxes1*boxes.max()\n        #scores = scores1\n        \n        boxes = boxes[scores >= detection_threshold].astype(np.int32)\n        scores = scores[scores >= detection_threshold]\n        image_id = image_ids[i]\n        \n        #boxes[:, 2] = boxes[:, 2] - boxes[:, 0]\n        #boxes[:, 3] 

In [ ]:
predictions_df = pd.DataFrame(results)
predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2848 entries, 0 to 2847
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   image_id       2848 non-null   object
 1   predictions    2848 non-null   object
 2   probabilities  2848 non-null   object
dtypes: object(3)
memory usage: 66.9+ KB


In [ ]:
predictions_df.head()

,image_id,predictions,probabilities
0,c087c8b89904edee56997a25d64f4cd1b7c9441e64889a...,253 374 312 445;804 595 885 637;850 302 922 38...,0.99920976;0.99824095;0.997666;0.99760926;0.99...
1,b11a617a64ca6120021e70b107933ddb70bf052395759d...,676 921 748 990;19 779 135 846;826 428 876 480...,0.9997247;0.99954516;0.9993236;0.99899095;0.99...
2,4fd7a4dc7dc60b5daedc7d57fdf8000ef3cab267654b56...,738 295 831 350;844 0 898 51;385 764 504 816;7...,0.99981457;0.99974054;0.9997118;0.9997019;0.99...
3,3f60fada17caf269a8d70798eb3bb5771f2c10d13c887e...,287 1 362 87;868 564 971 623;338 779 437 828;7...,0.99988484;0.9996357;0.9996165;0.99952173;0.99...
4,8cdcb28a926c217d0b587fcba2050e283b3175824a5b6e...,560 943 635 998;513 578 587 644;242 636 315 68...,0.9994708;0.99943954;0.99941456;0.99937326;0.9...


In [ ]:
predictions_df.to_csv('/content/drive/MyDrive/aicrowd_wheat/fasterrcnn_best_preds.csv', index=False)

In [ ]:
submission_df = test_df.merge(predictions_df, on=['image_id'])
assert submission_df.shape[0]==test_df.shape[0]
submission_df['PredString'] = submission_df['predictions']
submission_df = submission_df[["image_name","PredString","domain"]]
submission_df.head()

,image_name,PredString,domain
0,c087c8b89904edee56997a25d64f4cd1b7c9441e64889a...,253 374 312 445;804 595 885 637;850 302 922 38...,18
1,b11a617a64ca6120021e70b107933ddb70bf052395759d...,676 921 748 990;19 779 135 846;826 428 876 480...,18
2,4fd7a4dc7dc60b5daedc7d57fdf8000ef3cab267654b56...,738 295 831 350;844 0 898 51;385 764 504 816;7...,18
3,3f60fada17caf269a8d70798eb3bb5771f2c10d13c887e...,287 1 362 87;868 564 971 623;338 779 437 828;7...,18
4,8cdcb28a926c217d0b587fcba2050e283b3175824a5b6e...,560 943 635 998;513 578 587 644;242 636 315 68...,18


In [ ]:
submission_df.PredString.iloc[2]

'738 295 831 350;844 0 898 51;385 764 504 816;775 498 844 564;274 22 383 113;0 442 67 501;662 66 741 134;105 784 211 882;846 118 898 173;407 468 476 523;46 9 145 60;524 521 585 564;54 418 152 462;667 631 737 697;591 6 681 75;91 650 248 705;292 779 378 851;929 212 994 287;0 203 61 251;71 142 145 201;420 654 511 700;464 429 529 474;206 718 272 755;20 706 113 770;464 674 543 736;439 81 517 143;600 337 675 391;829 668 879 731;687 837 782 899;734 529 809 598;733 591 786 636;966 625 1020 677;2 767 118 813;933 93 997 141;0 665 46 719;929 739 987 782;379 322 439 416;616 693 682 785;45 507 174 565;12 47 129 98;484 206 551 244;350 608 438 662;299 416 389 466;863 449 907 493;942 703 999 742;195 439 310 504;985 428 1023 488;286 375 404 422;281 528 365 581;865 708 906 779;777 977 847 1022;296 480 368 523;382 595 505 640;632 158 674 196;908 970 965 1023;130 453 206 491;184 319 244 363'

In [ ]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2849 entries, 0 to 2848
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  2849 non-null   object
 1   PredString  2849 non-null   object
 2   domain      2849 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 89.0+ KB


In [ ]:
'''df_binary = pd.read_csv('/content/drive/MyDrive/aicrowd_wheat/binary_preds.csv')
df_binary['image_name'] = df_binary['file_name'].str.replace('.png','')
submission_check = submission_df.merge(df_binary, on=['image_name'])
assert submission_df.shape[0] == submission_check.shape[0]
submission_check['has_boxes'] = np.where(submission_check['PredString']=='no_box','no','yes')
submission_check['PredString'] = np.where(submission_check['prediction']=='no',
                                          'no_box', submission_check['PredString'])
submission_df = submission_check[["image_name","PredString","domain"]]
submission_df.info()'''

'df_binary = pd.read_csv(\'/content/drive/MyDrive/aicrowd_wheat/binary_preds.csv\')\ndf_binary[\'image_name\'] = df_binary[\'file_name\'].str.replace(\'.png\',\'\')\nsubmission_check = submission_df.merge(df_binary, on=[\'image_name\'])\nassert submission_df.shape[0] == submission_check.shape[0]\nsubmission_check[\'has_boxes\'] = np.where(submission_check[\'PredString\']==\'no_box\',\'no\',\'yes\')\nsubmission_check[\'PredString\'] = np.where(submission_check[\'prediction\']==\'no\',\n                                          \'no_box\', submission_check[\'PredString\'])\nsubmission_df = submission_check[["image_name","PredString","domain"]]\nsubmission_df.info()'

In [ ]:
# submission_check[(submission_check['has_boxes']!=submission_check['prediction'])  & (submission_check['prediction']=='no')]

In [ ]:
API_KEY = "9967b67c796b4d50e5d26f4dfd97ed06"
submission_df.to_csv("submission_final.csv", index=False)

In [ ]:
!pip install aicrowd-cli
!aicrowd login --api-key $API_KEY
!aicrowd submission create -c global-wheat-challenge-2021 -f submission_final.csv

     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 81kB 9.0MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 215kB 17.9MB/s 
     |████████████████████████████████| 174kB 20.0MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 71kB 11.9MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


API Key valid
Saved API Key successfully!
submission_final.csv ━━━━━━━━━━━━━━━━━━ 100.0% • 1.6/1.6 MB • 3.3 MB/s • 0:00:00
                                             ╭─────────────────────────╮                                             
                                             │ Successfully submitted! │                                             
                                             ╰─────────────────────────╯                                             
                                                   Important links                                                   
┌──────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/global-wheat-challenge-2021/submissions/147339              │
│                  │                                                                                                │
│  All submissions │ https://www.aicrowd.com/challe